# GPT-2 LayerNorm Validation

**Paper #3 - Critical Validation Test**

**Date:** 2026-01-05

**Purpose:** Confirm that LayerNorm models consistently show Gain < 1.0

---

## Hypothesis

If Pythia-6.9B (LayerNorm) shows Gain = 0.80, then GPT-2 (also LayerNorm) should also show Gain < 1.0.

This would confirm that **LayerNorm = Dampening** is a general property, not a Pythia-specific artifact.

| Model | Norm Type | Expected Gain | Status |
|-------|-----------|---------------|--------|
| Pythia-6.9B | LayerNorm | 0.80 | MEASURED |
| **GPT-2-XL** | **LayerNorm** | **< 1.0** | **TO TEST** |
| Mistral-7B | RMSNorm | 1.11 | MEASURED |
| LLaMA-3.1-8B | RMSNorm | 1.48 | MEASURED |
| Gemma-7B | RMSNorm | 2.31 | MEASURED |

---

## Models to Test

- **GPT-2 XL (1.5B)** - Large enough for meaningful comparison
- **GPT-2 Large (774M)** - Secondary validation
- **GPT-2 Medium (355M)** - Tertiary validation (optional)

In [ ]:
# Cell 1: Setup
!pip install -q transformers accelerate scipy seaborn

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
from scipy.stats import entropy, spearmanr, pearsonr, ttest_ind
import gc
import json
from datetime import datetime
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# HF Token handling - GPT-2 is public, but we set up the pattern for consistency
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("HF_TOKEN loaded from Colab secrets")
except:
    HF_TOKEN = None
    print("No HF_TOKEN (not needed for GPT-2)")

# Configure visualization
plt.style.use('seaborn-v0_8-paper')
sns.set_context("talk")

# Global timestamp for all files
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"\nSession timestamp: {TIMESTAMP}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 2: PROMPT SET (Same as Grand Unified Benchmark)

PROMPT_DATASET = {
    "Factual": [
        "The capital city of France is",
        "The atomic number of oxygen is",
        "Water boils at a temperature of",
        "The largest planet in our solar system is",
        "The currency used in Japan is"
    ],
    "Syntactic": [
        "The agreement, which, notwithstanding the fact that it was signed only yesterday, effectively binds all parties immediately, stipulates that",
        "Although the weather was extremely cold, and despite the fact that they had no coats, the children decided to",
        "The professor, having reviewed the complex derivation multiple times without finding the error, finally realized that",
        "To imply that such a fundamental shift in policy could occur without significant public debate is to suggest that",
        "Not only did the experiment fail to yield the expected results, but it also demonstrated that the initial hypothesis was"
    ],
    "Cliche": [
        "The true meaning of happiness is often found in",
        "Actions speak louder than",
        "It is what it is, and we must simply",
        "Time heals all",
        "Life is a journey, not a"
    ],
    "Novel": [
        "The epistemological implications of quantum decoherence suggest that the observer is",
        "If consciousness creates reality, then the paradox of the unobserved electron implies",
        "The intersection of baroque architecture and cybernetic theory creates a space where",
        "Calculating the trajectory of a hyperspace jump requires factoring in the variability of",
        "The symbiotic relationship between fungal mycelium and digital neural networks results in"
    ],
    "Nonsense": [
        "Table sky run blue jump quickly under over",
        "Purple idea furiously sleep colorless green",
        "Clock river dance potato seven fast",
        "Window eat loud tomorrow yellow under",
        "Fish bicycle logic cloud mountain swim"
    ]
}

CATEGORY_METADATA = {
    "Factual": {"expected_entropy": "medium", "complexity": 1},
    "Syntactic": {"expected_entropy": "medium", "complexity": 5},
    "Cliche": {"expected_entropy": "low", "complexity": 2},
    "Novel": {"expected_entropy": "high", "complexity": 4},
    "Nonsense": {"expected_entropy": "very_high", "complexity": 3}
}

total_prompts = sum(len(v) for v in PROMPT_DATASET.values())
print(f"Total prompts: {total_prompts} (5 categories x 5 prompts)")
for cat, prompts in PROMPT_DATASET.items():
    print(f"  {cat}: {len(prompts)} prompts")

In [ ]:
# Cell 3: GPT-2 MODEL VARIANTS (All LayerNorm)

MODELS_TO_TEST = {
    "GPT2-XL": {
        "hf_path": "gpt2-xl",
        "norm_type": "LayerNorm",
        "params": "1.5B",
        "expected_base": "< 1.0",
        "role": "LayerNorm Validation (Large)"
    },
    "GPT2-Large": {
        "hf_path": "gpt2-large",
        "norm_type": "LayerNorm",
        "params": "774M",
        "expected_base": "< 1.0",
        "role": "LayerNorm Validation (Medium)"
    },
    "GPT2-Medium": {
        "hf_path": "gpt2-medium",
        "norm_type": "LayerNorm",
        "params": "355M",
        "expected_base": "< 1.0",
        "role": "LayerNorm Validation (Small)"
    }
}

# Reference values from Grand Unified Benchmark
REFERENCE_MODELS = {
    "Pythia-6.9B": {"gain": 0.80, "norm": "LayerNorm"},
    "Mistral-7B": {"gain": 1.11, "norm": "RMSNorm"},
    "LLaMA-3.1-8B": {"gain": 1.48, "norm": "RMSNorm"},
    "Gemma-7B": {"gain": 2.31, "norm": "RMSNorm"}
}

print("GPT-2 Models to test (all LayerNorm):")
for name, info in MODELS_TO_TEST.items():
    print(f"  {name} ({info['params']}): {info['hf_path']}")

print("\nReference values from Grand Unified Benchmark:")
for name, info in REFERENCE_MODELS.items():
    print(f"  {name}: Gain={info['gain']:.2f} ({info['norm']})")

In [ ]:
# Cell 4: MEASUREMENT ENGINE (GPT-2 Compatible)

def get_layer_list(model):
    """Get the transformer layers from different model architectures."""
    # GPT-2 style: model.transformer.h
    if hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
        return model.transformer.h
    # LLaMA/Mistral/Gemma style: model.model.layers
    elif hasattr(model, 'model') and hasattr(model.model, 'layers'):
        return model.model.layers
    # Pythia style: model.gpt_neox.layers
    elif hasattr(model, 'gpt_neox') and hasattr(model.gpt_neox, 'layers'):
        return model.gpt_neox.layers
    else:
        raise ValueError(f"Unknown model architecture: {type(model)}")

def measure_thermodynamics(model, tokenizer, text, device='cuda'):
    """Measure residual stream gain and output entropy for a given input."""
    
    # Tokenize - handle padding token for GPT-2
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Hooks to capture residual stream norms
    norms = []
    
    def get_norm_hook():
        def hook(module, input, output):
            # Handle different output formats
            if isinstance(output, tuple):
                hidden_state = output[0]
            else:
                hidden_state = output
            
            # Ensure we're working with a tensor
            if hasattr(hidden_state, 'last_hidden_state'):
                hidden_state = hidden_state.last_hidden_state
            
            # Norm of last token - convert to float explicitly
            try:
                last_token_norm = float(torch.norm(hidden_state[0, -1]).cpu().item())
                norms.append(last_token_norm)
            except Exception as e:
                print(f"Warning: Could not compute norm: {e}")
        return hook
    
    # Register hooks on all layers
    handles = []
    try:
        layers = get_layer_list(model)
        for layer in layers:
            handles.append(layer.register_forward_hook(get_norm_hook()))
    except Exception as e:
        print(f"Warning: Could not register hooks: {e}")
        return None
    
    # Forward pass
    try:
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
    except Exception as e:
        print(f"Error in forward pass: {e}")
        for h in handles:
            h.remove()
        return None
    
    # Cleanup hooks
    for h in handles:
        h.remove()
    
    # Calculate metrics - explicit float conversion to avoid numpy issues
    if len(norms) >= 2:
        # Last Layer Gain = Output of Last Layer / Output of Penultimate Layer
        last_gain = float(norms[-1] / norms[-2]) if norms[-2] > 0 else 1.0
        # Total Amplification = Final / Initial
        total_amp = float(norms[-1] / norms[0]) if norms[0] > 0 else 1.0
    else:
        last_gain = 1.0
        total_amp = 1.0
    
    # Output Entropy (next token distribution)
    last_token_logits = logits[0, -1, :].float()  # Ensure float32
    probs = torch.softmax(last_token_logits, dim=0).cpu().numpy().astype(np.float64)
    
    # Clip to avoid log(0) issues
    probs = np.clip(probs, 1e-10, 1.0)
    probs = probs / probs.sum()  # Renormalize
    
    ent = float(entropy(probs))
    
    # Top token and probability
    top_idx = int(torch.argmax(last_token_logits).item())
    top_prob = float(probs[top_idx])
    top_token = tokenizer.decode([top_idx])
    
    return {
        "last_gain": float(last_gain),
        "total_amp": float(total_amp),
        "entropy": float(ent),
        "top_token": str(top_token),
        "top_prob": float(top_prob),
        "n_layers": int(len(norms)),
        "all_norms": [float(n) for n in norms]
    }

print("Measurement engine ready (GPT-2 compatible).")

In [ ]:
# Cell 5: EXECUTION LOOP

all_results = []

print("=" * 70)
print("STARTING GPT-2 LAYERNORM VALIDATION")
print(f"Models: {len(MODELS_TO_TEST)} | Prompts: {total_prompts}")
print(f"Total measurements: {len(MODELS_TO_TEST) * total_prompts}")
print("=" * 70 + "\n")

for model_name, model_info in MODELS_TO_TEST.items():
    hf_path = model_info["hf_path"]
    print(f"\n{'='*60}")
    print(f"Loading {model_name} ({model_info['params']})...")
    print(f"  Path: {hf_path}")
    print(f"  Norm: {model_info['norm_type']}")
    print(f"  Expected Base: {model_info['expected_base']}")
    
    try:
        # Load tokenizer
        tokenizer = GPT2Tokenizer.from_pretrained(hf_path)
        tokenizer.pad_token = tokenizer.eos_token
        
        # Load model with explicit dtype
        model = GPT2LMHeadModel.from_pretrained(
            hf_path,
            torch_dtype=torch.float32,  # GPT-2 works better with float32
            device_map="auto"
        )
        model.eval()
        
        print(f"\n  Testing {len(PROMPT_DATASET)} categories ({total_prompts} prompts)...")
        
        for category, prompts in PROMPT_DATASET.items():
            print(f"    {category}: ", end="")
            for i, prompt in enumerate(prompts):
                res = measure_thermodynamics(model, tokenizer, prompt)
                
                if res is not None:
                    all_results.append({
                        "Model": str(model_name),
                        "Norm_Type": str(model_info["norm_type"]),
                        "Params": str(model_info["params"]),
                        "Role": str(model_info["role"]),
                        "Category": str(category),
                        "Complexity": int(CATEGORY_METADATA[category]["complexity"]),
                        "Prompt": str(prompt),
                        "Prompt_Short": str(prompt[:40] + "..."),
                        "Entropy": float(res["entropy"]),
                        "Last_Gain": float(res["last_gain"]),
                        "Total_Amp": float(res["total_amp"]),
                        "Top_Token": str(res["top_token"]),
                        "Top_Prob": float(res["top_prob"]),
                        "N_Layers": int(res["n_layers"])
                    })
                    print(".", end="", flush=True)
                else:
                    print("X", end="", flush=True)
            print(f" Done (n={len(prompts)})")
        
        # Cleanup
        del model
        del tokenizer
        torch.cuda.empty_cache()
        gc.collect()
        print(f"\n  {model_name} complete. Memory cleared.")
        
    except Exception as e:
        print(f"\n  FAILED: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "=" * 70)
print("BENCHMARK COMPLETE")
print(f"Total measurements: {len(all_results)}")
print("=" * 70)

In [ ]:
# Cell 6: CREATE DATAFRAME & SUMMARY

df = pd.DataFrame(all_results)

# Define filenames with global timestamp
CSV_FILE = f"gpt2_layernorm_validation_{TIMESTAMP}.csv"
JSON_FILE = f"gpt2_layernorm_validation_{TIMESTAMP}.json"
PNG_MAIN = f"gpt2_layernorm_validation_{TIMESTAMP}.png"
PNG_COMPARISON = f"gpt2_vs_reference_{TIMESTAMP}.png"

# Save raw results
df.to_csv(CSV_FILE, index=False)
print(f"Saved: {CSV_FILE}")

# Display summary
print("\n" + "=" * 70)
print("GPT-2 RESULTS SUMMARY")
print("=" * 70)

print("\nMean Gain per Model:")
print(df.groupby('Model')['Last_Gain'].agg(['mean', 'std', 'min', 'max']).round(3))

print("\nMean Gain per Category:")
print(df.groupby(['Model', 'Category'])['Last_Gain'].mean().unstack().round(3))

In [ ]:
# Cell 7: CRITICAL VALIDATION - LayerNorm Hypothesis

print("\n" + "=" * 70)
print("CRITICAL VALIDATION: LAYERNORM DAMPENING HYPOTHESIS")
print("=" * 70)

print("\nHypothesis: All LayerNorm models show Gain < 1.0 (dampening)")
print("\nReference: Pythia-6.9B (LayerNorm) = 0.80")
print("-" * 60)

# Calculate results
validation_results = []

for model in df['Model'].unique():
    subset = df[df['Model'] == model]
    mean_gain = float(subset['Last_Gain'].mean())
    std_gain = float(subset['Last_Gain'].std())
    min_gain = float(subset['Last_Gain'].min())
    max_gain = float(subset['Last_Gain'].max())
    
    # Hypothesis test: is mean significantly < 1.0?
    from scipy.stats import ttest_1samp
    t_stat, p_val = ttest_1samp(subset['Last_Gain'], 1.0)
    
    # One-sided p-value (we expect < 1.0)
    p_one_sided = float(p_val / 2) if t_stat < 0 else float(1 - p_val / 2)
    
    is_dampening = mean_gain < 1.0
    is_significant = p_one_sided < 0.05 and is_dampening
    
    status = "CONFIRMED" if is_significant else "NOT SIGNIFICANT" if is_dampening else "REJECTED"
    
    validation_results.append({
        "Model": model,
        "Mean_Gain": mean_gain,
        "Std": std_gain,
        "Min": min_gain,
        "Max": max_gain,
        "t_stat": float(t_stat),
        "p_value": p_one_sided,
        "Is_Dampening": bool(is_dampening),
        "Status": status
    })
    
    sig_marker = "***" if p_one_sided < 0.001 else "**" if p_one_sided < 0.01 else "*" if p_one_sided < 0.05 else ""
    
    print(f"\n{model}:")
    print(f"  Mean Gain: {mean_gain:.3f} +/- {std_gain:.3f}")
    print(f"  Range: [{min_gain:.3f}, {max_gain:.3f}]")
    print(f"  t-test vs 1.0: t={t_stat:.3f}, p={p_one_sided:.4f} {sig_marker}")
    print(f"  Status: {status}")

# Overall verdict
print("\n" + "=" * 70)
print("OVERALL VERDICT")
print("=" * 70)

all_dampening = all(r["Is_Dampening"] for r in validation_results)
all_significant = all(r["Status"] == "CONFIRMED" for r in validation_results)

if all_significant:
    print("\n  LAYERNORM DAMPENING HYPOTHESIS: STRONGLY CONFIRMED")
    print("  All GPT-2 variants show statistically significant Gain < 1.0")
elif all_dampening:
    print("\n  LAYERNORM DAMPENING HYPOTHESIS: CONFIRMED (directionally)")
    print("  All GPT-2 variants show Gain < 1.0 (not all statistically significant)")
else:
    print("\n  LAYERNORM DAMPENING HYPOTHESIS: INCONCLUSIVE")
    print("  Some GPT-2 variants do not show clear dampening")

# Comparison with Pythia
gpt2_xl_gain = df[df['Model'] == 'GPT2-XL']['Last_Gain'].mean() if 'GPT2-XL' in df['Model'].values else None
pythia_gain = 0.80  # Reference value

if gpt2_xl_gain is not None:
    print(f"\n  GPT2-XL vs Pythia-6.9B:")
    print(f"    GPT2-XL:    {gpt2_xl_gain:.3f}")
    print(f"    Pythia-6.9B: {pythia_gain:.3f}")
    print(f"    Difference: {abs(gpt2_xl_gain - pythia_gain):.3f}")
    
    if abs(gpt2_xl_gain - pythia_gain) < 0.15:
        print(f"    --> CONSISTENT (both in dampening regime)")
    else:
        print(f"    --> DIVERGENT (different dampening magnitude)")

In [ ]:
# Cell 8: VISUALIZATION - Main Results

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# A. Gain Distribution per GPT-2 Model
ax1 = axes[0, 0]
for model in df['Model'].unique():
    subset = df[df['Model'] == model]
    ax1.hist(subset['Last_Gain'], bins=15, alpha=0.6, label=model)
ax1.axvline(1.0, ls='--', c='red', lw=2, label='Neutral (1.0)')
ax1.axvline(0.80, ls=':', c='blue', lw=2, label='Pythia Ref (0.80)')
ax1.set_xlabel('Last Layer Gain')
ax1.set_ylabel('Count')
ax1.set_title('A. Gain Distribution (GPT-2 LayerNorm Models)')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# B. Scatter: Entropy vs Gain
ax2 = axes[0, 1]
for model in df['Model'].unique():
    subset = df[df['Model'] == model]
    ax2.scatter(subset['Entropy'], subset['Last_Gain'], label=model, alpha=0.7, s=80)
ax2.axhline(1.0, ls='--', c='red', alpha=0.5, label='Neutral (1.0)')
ax2.set_xlabel('Output Entropy (nats)')
ax2.set_ylabel('Last Layer Gain')
ax2.set_title('B. Entropy vs Gain (GPT-2 Models)')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# C. Box: Gain by Category
ax3 = axes[1, 0]
category_order = ['Factual', 'Cliche', 'Nonsense', 'Novel', 'Syntactic']
sns.boxplot(data=df, x='Category', y='Last_Gain', hue='Model', ax=ax3, order=category_order)
ax3.axhline(1.0, ls='--', c='red', alpha=0.5)
ax3.axhline(0.80, ls=':', c='blue', alpha=0.5)
ax3.set_xlabel('Prompt Category')
ax3.set_ylabel('Last Layer Gain')
ax3.set_title('C. Gain by Category (GPT-2)')
ax3.legend(loc='upper right', fontsize=8)
ax3.tick_params(axis='x', rotation=45)

# D. Base Level Bar Chart with Reference
ax4 = axes[1, 1]

# GPT-2 models
gpt2_means = df.groupby('Model')['Last_Gain'].mean().sort_values()
gpt2_stds = df.groupby('Model')['Last_Gain'].std()

# Combined data for plotting
all_models = list(gpt2_means.index) + list(REFERENCE_MODELS.keys())
all_means = list(gpt2_means.values) + [v['gain'] for v in REFERENCE_MODELS.values()]
all_norms = ['LayerNorm'] * len(gpt2_means) + [v['norm'] for v in REFERENCE_MODELS.values()]

# Colors: Blue for LayerNorm, Orange for RMSNorm
colors = ['#1f77b4' if n == 'LayerNorm' else '#ff7f0e' for n in all_norms]

bars = ax4.bar(range(len(all_models)), all_means, color=colors, alpha=0.7)
ax4.axhline(1.0, ls='--', c='red', alpha=0.5, label='Neutral')
ax4.set_xticks(range(len(all_models)))
ax4.set_xticklabels(all_models, rotation=45, ha='right')
ax4.set_xlabel('Model')
ax4.set_ylabel('Mean Last Layer Gain')
ax4.set_title('D. GPT-2 vs Reference Models\n(Blue=LayerNorm, Orange=RMSNorm)')

# Add value labels
for bar, val in zip(bars, all_means):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(PNG_MAIN, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {PNG_MAIN}")

In [ ]:
# Cell 9: VISUALIZATION - LayerNorm vs RMSNorm Comparison

fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Prepare data
layernorm_models = []
rmsnorm_models = []

# GPT-2 models (all LayerNorm)
for model in df['Model'].unique():
    gain = float(df[df['Model'] == model]['Last_Gain'].mean())
    layernorm_models.append((model, gain, 'GPT-2'))

# Pythia (LayerNorm)
layernorm_models.append(('Pythia-6.9B', 0.80, 'Reference'))

# RMSNorm reference models
rmsnorm_models = [
    ('Mistral-7B', 1.11, 'Reference'),
    ('LLaMA-3.1-8B', 1.48, 'Reference'),
    ('Gemma-7B', 2.31, 'Reference')
]

# Plot LayerNorm models
ln_names = [m[0] for m in layernorm_models]
ln_gains = [m[1] for m in layernorm_models]
ln_colors = ['#1f77b4' if m[2] == 'GPT-2' else '#17becf' for m in layernorm_models]

# Plot RMSNorm models
rms_names = [m[0] for m in rmsnorm_models]
rms_gains = [m[1] for m in rmsnorm_models]

# Combined plot
all_names = ln_names + rms_names
all_gains = ln_gains + rms_gains
all_colors = ln_colors + ['#ff7f0e'] * len(rms_names)

# Sort by gain
sorted_data = sorted(zip(all_names, all_gains, all_colors), key=lambda x: x[1])
all_names = [d[0] for d in sorted_data]
all_gains = [d[1] for d in sorted_data]
all_colors = [d[2] for d in sorted_data]

bars = ax.barh(range(len(all_names)), all_gains, color=all_colors, alpha=0.8)
ax.axvline(1.0, ls='--', c='red', lw=2, label='Neutral (1.0)')
ax.set_yticks(range(len(all_names)))
ax.set_yticklabels(all_names)
ax.set_xlabel('Mean Last Layer Gain')
ax.set_title('LayerNorm (Blue) vs RMSNorm (Orange)\nThe Dampening Hypothesis', fontsize=14)

# Add value labels
for bar, val in zip(bars, all_gains):
    ax.text(val + 0.03, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', ha='left', va='center', fontsize=11, fontweight='bold')

# Add annotations
ax.annotate('DAMPENING\n(Gain < 1.0)', xy=(0.5, 0.3), xycoords='axes fraction',
            fontsize=12, ha='center', color='blue', alpha=0.7)
ax.annotate('EXPANSION\n(Gain > 1.0)', xy=(0.8, 0.7), xycoords='axes fraction',
            fontsize=12, ha='center', color='orange', alpha=0.7)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='GPT-2 (LayerNorm) - This Test'),
    Patch(facecolor='#17becf', label='Pythia (LayerNorm) - Reference'),
    Patch(facecolor='#ff7f0e', label='RMSNorm Models - Reference')
]
ax.legend(handles=legend_elements, loc='lower right')

ax.grid(True, alpha=0.3, axis='x')
ax.set_xlim(0, max(all_gains) * 1.15)

plt.tight_layout()
plt.savefig(PNG_COMPARISON, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {PNG_COMPARISON}")

In [ ]:
# Cell 10: ENTROPY CORRELATION ANALYSIS (Bentov Law Test)

print("\n" + "=" * 70)
print("BENTOV LAW TEST: |Gain - 1.0| vs Entropy")
print("=" * 70)

print("\nHypothesis: Pythia showed NEGATIVE correlation (r = -0.20)")
print("If GPT-2 also shows negative correlation, LayerNorm inverts the physics.")
print("-" * 60)

bentov_results = []

for model in df['Model'].unique():
    subset = df[df['Model'] == model].copy()
    
    # Calculate Bentov Deviation
    subset['Bentov_Deviation'] = abs(subset['Last_Gain'] - 1.0)
    
    # Correlation
    r, p = pearsonr(subset['Entropy'], subset['Bentov_Deviation'])
    
    sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
    
    bentov_results.append({
        "Model": model,
        "Bentov_Correlation": float(r),
        "p_value": float(p),
        "Direction": "Positive" if r > 0 else "Negative"
    })
    
    print(f"\n{model}:")
    print(f"  Correlation (Entropy vs |Gain-1|): r = {r:+.3f} {sig}")
    print(f"  p-value: {p:.4f}")
    print(f"  Direction: {'POSITIVE (RMSNorm-like)' if r > 0 else 'NEGATIVE (LayerNorm-like)'}")

# Compare with Pythia reference
print("\n" + "-" * 60)
print("Comparison with Reference:")
print(f"  Pythia-6.9B (LayerNorm): r = -0.199 (NEGATIVE)")
print(f"  Gemma-7B (RMSNorm):      r = +0.692 (POSITIVE)")
print("\nGPT-2 Results:")
for br in bentov_results:
    print(f"  {br['Model']}: r = {br['Bentov_Correlation']:+.3f} ({br['Direction']})")

# Verdict
gpt2_negative = sum(1 for br in bentov_results if br['Bentov_Correlation'] < 0)
if gpt2_negative == len(bentov_results):
    print("\n  --> ALL GPT-2 models show NEGATIVE correlation (like Pythia)")
    print("  --> LAYERNORM INVERTS THE BENTOV LAW: CONFIRMED")
elif gpt2_negative > 0:
    print(f"\n  --> {gpt2_negative}/{len(bentov_results)} GPT-2 models show negative correlation")
    print("  --> PARTIAL CONFIRMATION of LayerNorm inversion")
else:
    print("\n  --> GPT-2 models show POSITIVE correlation (unlike Pythia)")
    print("  --> LayerNorm inversion NOT confirmed for GPT-2")

In [ ]:
# Cell 11: SAVE ALL RESULTS AS JSON

# Helper function to ensure JSON serializable
def make_serializable(obj):
    if isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(v) for v in obj]
    return obj

results_json = {
    "experiment": "GPT-2 LayerNorm Validation",
    "purpose": "Confirm LayerNorm dampening hypothesis (n=2)",
    "date": TIMESTAMP,
    "n_models": len(MODELS_TO_TEST),
    "n_prompts": total_prompts,
    "n_measurements": len(df),
    "models_tested": list(MODELS_TO_TEST.keys()),
    "reference_models": REFERENCE_MODELS,
    "base_levels": {k: float(v) for k, v in df.groupby('Model')['Last_Gain'].mean().to_dict().items()},
    "validation_results": make_serializable(validation_results),
    "bentov_law_results": make_serializable(bentov_results),
    "hypothesis_status": {
        "layernorm_dampening": "CONFIRMED" if all(r["Is_Dampening"] for r in validation_results) else "REJECTED",
        "bentov_inversion": "CONFIRMED" if all(br['Bentov_Correlation'] < 0 for br in bentov_results) else "PARTIAL" if any(br['Bentov_Correlation'] < 0 for br in bentov_results) else "REJECTED"
    },
    "all_results": make_serializable(all_results)
}

with open(JSON_FILE, 'w') as f:
    json.dump(results_json, f, indent=2, default=str)

print(f"Results saved to {JSON_FILE}")

In [ ]:
# Cell 12: FINAL VERDICT FOR PAPER #3

print("\n" + "=" * 70)
print("FINAL VERDICT: LAYERNORM HYPOTHESIS VALIDATION")
print("=" * 70)

# Calculate key metrics
gpt2_models = list(df['Model'].unique())
gpt2_gains = {m: float(df[df['Model'] == m]['Last_Gain'].mean()) for m in gpt2_models}

print("\n" + "="*70)
print("LAYERNORM MODELS (n=4 including Pythia reference)")
print("="*70)
print(f"\n  Pythia-6.9B:   {0.80:.2f}  (Reference)")
for model, gain in sorted(gpt2_gains.items(), key=lambda x: x[1]):
    print(f"  {model}:   {gain:.2f}  (This Test)")

all_layernorm_gains = [0.80] + list(gpt2_gains.values())
mean_layernorm = np.mean(all_layernorm_gains)
std_layernorm = np.std(all_layernorm_gains)

print(f"\n  LayerNorm Mean: {mean_layernorm:.3f} +/- {std_layernorm:.3f}")
print(f"  All < 1.0: {all(g < 1.0 for g in all_layernorm_gains)}")

print("\n" + "="*70)
print("RMSNORM MODELS (Reference from Grand Unified Benchmark)")
print("="*70)
print(f"\n  Mistral-7B:    {1.11:.2f}")
print(f"  LLaMA-3.1-8B:  {1.48:.2f}")
print(f"  Gemma-7B:      {2.31:.2f}")

rmsnorm_gains = [1.11, 1.48, 2.31]
mean_rmsnorm = np.mean(rmsnorm_gains)
print(f"\n  RMSNorm Mean: {mean_rmsnorm:.3f}")
print(f"  All > 1.0: {all(g > 1.0 for g in rmsnorm_gains)}")

print("\n" + "="*70)
print("CONCLUSION FOR PAPER #3")
print("="*70)

if all(g < 1.0 for g in all_layernorm_gains) and all(g > 1.0 for g in rmsnorm_gains):
    print("""
  ┌─────────────────────────────────────────────────────────────┐
  │  LAYERNORM VS RMSNORM DICHOTOMY: STRONGLY CONFIRMED        │
  ├─────────────────────────────────────────────────────────────┤
  │                                                             │
  │  LayerNorm (n=4): ALL show Gain < 1.0 (Dampening)          │
  │  RMSNorm (n=3):   ALL show Gain > 1.0 (Expansion)          │
  │                                                             │
  │  This is NOT a Pythia-specific artifact.                   │
  │  This is a FUNDAMENTAL PROPERTY of the normalization.      │
  │                                                             │
  │  Paper #3 Claim: VALIDATED (n=2 for LayerNorm)             │
  │                                                             │
  └─────────────────────────────────────────────────────────────┘
""")
else:
    print("\n  Results are mixed. See detailed analysis above.")

print("\n" + "="*70)

In [ ]:
# Cell 13: DOWNLOAD ALL RESULTS

print("\n" + "=" * 70)
print("DOWNLOADING RESULTS")
print("=" * 70)

# List all files to download
files_to_download = [CSV_FILE, JSON_FILE, PNG_MAIN, PNG_COMPARISON]

print("\nFiles to download:")
for f in files_to_download:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024  # KB
        print(f"  {f} ({size:.1f} KB)")
    else:
        print(f"  {f} (NOT FOUND)")

print("\nStarting downloads...")

# Download each file
for f in files_to_download:
    if os.path.exists(f):
        try:
            files.download(f)
            print(f"  Downloaded: {f}")
        except Exception as e:
            print(f"  FAILED to download {f}: {e}")
    else:
        print(f"  SKIPPED (not found): {f}")

print("\n" + "=" * 70)
print("ALL DOWNLOADS COMPLETE")
print("=" * 70)
print("\nNext step: Add these results to Paper #3 documentation.")